In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

In [3]:
ticker = 'SPY'
data = yf.download(ticker, start = '2000-01-01', end = '2023-09-13')

[*********************100%***********************]  1 of 1 completed


In [5]:
data['Daily Ret'] = data['Adj Close'].pct_change()
data['State'] = np.where(data['Daily Ret'] >= 0, 'up', 'down')

In [6]:
data

,Open,High,Low,Close,Adj Close,Volume,Daily Ret,State
Date,,,,,,,,
2000-01-03,148.250000,148.250000,143.875000,145.437500,94.595055,8164300,NaN,down
2000-01-04,143.531250,144.062500,139.640625,139.750000,90.895798,8089800,-0.039106,down
2000-01-05,139.937500,141.531250,137.250000,140.000000,91.058418,12177900,0.001789,up
2000-01-06,139.625000,141.500000,137.750000,137.750000,89.594963,6227200,-0.016072,down
2000-01-07,140.312500,145.750000,140.062500,145.750000,94.798294,8066500,0.058076,up
...,...,...,...,...,...,...,...,...
2023-09-06,448.399994,448.510010,443.809998,446.220001,446.220001,70758500,-0.006722,down
2023-09-07,443.109985,445.549988,442.750000,444.850006,444.850006,70355400,-0.003070,down
2023-09-08,444.899994,447.109985,444.529999,445.519989,445.519989,61659700,0.001506,up


In [29]:
up_len = len(data[data['State'] == 'up'])
down_len = len(data[data['State'] == 'down'])

up = data['State'] == 'up'
down = data['State'] == 'down'

up_to_up = len(data[(data['State'] == 'up') & (data['State'].shift(-1) == 'up')]) / len(data.query('State == "up"'))
up_to_down = len(data[(data['State'] == 'down') & (data['State'].shift(-1) == 'up')]) / len(data.query('State == "down"')) # up day followed by an down day
down_to_down = len(data[(data['State'] == 'down') & (data['State'].shift(-1) == 'down')]) / len(data.query('State == "down"'))
down_to_up = len(data[(data['State'] == 'up') & (data['State'].shift(-1) == 'down')]) / len(data.query('State == "up"')) # down day followed by an up day

print(f'up to up: {up_to_up}')
print(f'up to down: {up_to_down}')
print(f'down to up: {down_to_up}')
print(f'down to down: {down_to_down}')

up to up: 0.533394890735611
up to down: 0.5589970501474927
down to up: 0.46660510926438903
down to down: 0.44063421828908556


In [30]:
trans_matrix = pd.DataFrame({'up': [up_to_up, down_to_up], 'down': [up_to_down, down_to_down]}, index = ['up', 'down'])
print(trans_matrix)

            up      down
up    0.533395  0.558997
down  0.466605  0.440634


In [35]:
prob_up2down = len(data[(up) & (down.shift(-1)) & (down.shift(-2)) & (down.shift(-3)) & (down.shift(-4)) & (down.shift(-5))]) / len(data[(down.shift(1)) & (down.shift(2)) & (down.shift(3)) & (down.shift(4)) & (down.shift(5))])
print(f'Probability of and up day followed by 5 consecutive down days: {prob_up2down}')

Probability of and up day followed by 5 consecutive down days: 0.68
